In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
sns.set()

In [2]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [3]:
with open('Flight2_HKG.txt', mode='r') as BabyFlight:
    Flight1 = BabyFlight.read().split("\n")[1]

In [4]:
from scipy import spatial
from ipynb.fs.full.GetTrajectory import GetTrajectory
from ipynb.fs.full.PerpendicularDistance import NearestPointOnLine
from ipynb.fs.full.PerpendicularDistance import GetPerpendicularDistance
from ipynb.fs.full.PerpendicularDistance import EdgeSquareSearch, EdgeCircleSearch
from ipynb.fs.full.PerpendicularDistance import Haversine, FilterTrajectory, EdgeCovered, GetDirection1, GetHeading, GetAlpha, GetBeta, GetDistanceChange, GetHeadingIncrement, GetDirection2
from ipynb.fs.full.PerpendicularDistance import GetEdgesBetween, GetEdgeDistance, GetCorrectedEdge, GetCorrectedEdge2, GetEdgeDistance2, GetDistanceError, GetNeighbor, GetConnectivity, IsSublist


In [5]:
FlightTrajectoryNodes = GetTrajectory(Flight1)


In [6]:
from ipynb.fs.full.GetTaxiway import GetTaxiway
from ipynb.fs.full.GetTaxiway import GetEdgeID
from ipynb.fs.full.GetTaxiway import GetEdgeStart
from ipynb.fs.full.GetTaxiway import GetEdgeEnd
from ipynb.fs.full.GetTaxiway import GetEdgeLength
from ipynb.fs.full.GetTaxiway import GetEdgeSpecification
from ipynb.fs.full.GetTaxiway import GetEdgesAndNodes, GetNodeLinks

In [7]:
from ipynb.fs.full.FuzzyLogic import SMP1FuzzyLogic
from ipynb.fs.full.FuzzyLogic import SMP2FuzzyLogic


In [8]:
#Setting up some lists to sort through the data
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]
AllNodeLinks = GetNodeLinks(FlightNodesList, FlightEdgesList)

In [9]:
EdgesAndNodes = GetEdgesAndNodes(FlightEdgesList, FlightNodesList)
#EdgeID[0], NodeStart[1], NodeFinish[2], EdgeLength[3], EdgeSpecification[4], StartX[5], StartY[6], EndX[7], EndY[8]
EdgesAndNodes;

In [10]:
#Making arrays of lists for easier searching
GateEdges = []
for item in EdgesAndNodes:
    if item[4] == 'gate':
        GateEdges.append(item)

GateNodes = []
Node = []
Nodes = []
for item in FlightNodesList:
    Nodes.append((float(item[1]),float(item[2])))
for item in FlightTrajectoryNodes:
    Node.append((float(item[0]),float(item[1])))
for item in GateEdges:
    GateNodes.append((item[5], item[6]))
    GateNodes.append((item[7], item[8]))
    
tree = spatial.KDTree(Nodes) #This uses spatial from scipy


In [11]:
G = nx.Graph()

Nodict = {}
for index, item in enumerate(Node):
    NodeX = item[0]
    NodeY = item[1]
    Nodict[int(index+1)] = (NodeX, NodeY)
    
MapNodes = {}

for item in FlightNodesList:
    NodeX = float(item[1])
    NodeY = float(item[2])
    MapNodes[int(item[0])] = (NodeX, NodeY)

G.add_nodes_from(MapNodes.keys())
for n, p in iter(MapNodes.items()):
    G.node[n]['pos'] = p

NodeLinks = []
for item in EdgesAndNodes:
    NodeLinks.append((item[1], item[2], {'weight':item[3]}))
G.add_edges_from(NodeLinks)

#specification = nx.get_edge_attributes(G, 'specification')
weight = nx.get_edge_attributes(G, 'weight')
pos = nx.get_node_attributes(G, 'pos')

In [12]:
Node = FilterTrajectory(Node, 22.3, 113.9185, 0.1)
Node;

In [13]:
#Check which end has a gate nearest
#Check The Start using KDTree array find the closest Gate Nodes to the trajectory node
tree = spatial.KDTree(GateNodes)
HighlightedGateNodeIDs = list(tree.query(Node[0], 5)[1])
HighlightedStartGateNodes = []
for n in HighlightedGateNodeIDs:
        HighlightedStartGateNodes.append(GateNodes[n])

PointX = Node[0][0]
PointY = Node[0][1]
HighlightStartGateEdge = []
for m in GateEdges:
        for i in HighlightedStartGateNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightStartGateEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightStartGateEdge.append(m)
        else:
            pass
HighlightedStartGateEdges = list(set(HighlightStartGateEdge))
StartGateEdgesSuccess = []

for index, edge in enumerate(HighlightedStartGateEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
            StartGateEdgesSuccess.append((index, EdgeID, EdgeStart, EdgeEnd, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
            ChosenStartEdge = min(StartGateEdgesSuccess, key = lambda t: t[-1])
        else:
            print('This gate edge is not near')

#Check the end
HighlightedGateNodeIDs = list(tree.query(Node[-1], 5)[1])
HighlightedEndGateNodes = []
for n in HighlightedGateNodeIDs:
        HighlightedEndGateNodes.append(GateNodes[n])  
PointX = Node[-1][0]
PointY = Node[-1][1]
HighlightEndGateEdge = []
for m in GateEdges:
        for i in HighlightedEndGateNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEndGateEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEndGateEdge.append(m)
        else:
            pass

HighlightedEndGateEdges = list(set(HighlightEndGateEdge))

EndGateEdgesSuccess = []
for index, edge in enumerate(HighlightedEndGateEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            print(edge)
            print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance)
            EndGateEdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
            ChosenEndEdge = min(EndGateEdgesSuccess, key = lambda t: t[-1])
        else:
            print('This gate edge is not near')

#Reverse the Trajectory Nodes if a gate is found at the end
try:
    if ChosenEndEdge == []:
        StartEdge = ChosenStartEdge
    else:
        pass
except NameError:
    StartEdge = ChosenStartEdge
    ChosenEndEdge = []
try:
    if ChosenStartEdge == []:
        StartEdge = ChosenEndEdge
    else:
        pass
except NameError:
    StartEdge = ChosenEndEdge
    ChosenStartEdge = []
try:
    if ChosenEndEdge[-1] > ChosenStartEdge[-1]:
        StartEdge = ChosenStartEdge
except IndexError:
    pass
try:
    if ChosenStartEdge[-1] > ChosenEndEdge[-1]:
        StartEdge = ChosenEndEdge
except IndexError:
    pass

if StartEdge == ChosenEndEdge:
    NewNode = list(reversed(Node))
else:
    NewNode = Node

#If neither start nor finish are near a gate, then choose closest Edge

#Search for 5 closest EdgeNodes to Node[0]
if ChosenStartEdge == [] and ChosenEndEdge == []:
        tree = spatial.KDTree(Nodes)
        HighlightedNodeIDs = list(tree.query(NewNode[0], 5)[1])
        HighlightedNodes = []
        for n in HighlightedNodeIDs:
            HighlightedNodes.append(Nodes[n])  

        HighlightEdge = []
        for m in EdgesAndNodes:
            for i in HighlightedNodes:
                if m[5] == i[0] and m[6] == i[1]:
                    HighlightEdge.append(m)
                elif m[7] == i[0] and m[8] == i[1]:
                    HighlightEdge.append(m)
            else:
                pass
        HighlightedEdges = list(set(HighlightEdge))



        #Iterate through the Edges for each Trajectory Node
        for index, edge in enumerate(HighlightedEdges):
            EdgeID = edge[0]
            EdgeStart = edge[1]
            EdgeEnd = edge[2]
            EdgeLength = edge[3]
            EdgeType = edge[4]

            StartX = edge[5]
            StartY = edge[6]
            EndX = edge[7]
            EndY = edge[8]
            #If Edge is within radius r0 of Node, then return True
            if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.001) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
                (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
                (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
                PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
                (NewStartX, NewStartY, NewEndX, NewEndY) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
                print(NewStartX, NewStartY, NewEndX, NewEndY)
                Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
                if counter == 0:
                    pass
                print(edge)
                print(index, EdgeID, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance, Percentage)
                EndGateEdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), PDistance))
                ChosenEdge = min(EndGateEdgesSuccess, key = lambda t: t[-1])
            else:
                pass
            StartEdge = ChosenEdge
else:
    pass

This gate edge is not near
This gate edge is not near
This gate edge is not near
This gate edge is not near
This gate edge is not near
(2497, 2211, 2212, 132.4110791440937, 'gate', 22.3143411, 113.9300992, 22.3154685, 113.9296821)
0 2497 22.3143411 113.9300992 22.3154685 113.9296821 (22.314350403627685, 113.93009575797134) 0.0008484717113462358
(2542, 2289, 2290, 137.72245475253408, 'gate', 22.3132837, 113.9296113, 22.3121182, 113.930067)
1 2542 22.3132837 113.9296113 22.3121182 113.930067 (22.3132837, 113.9296113) 0.0008326793980916912
This gate edge is not near
(2496, 2209, 2210, 132.090974445492, 'gate', 22.3141092, 113.9293734, 22.3152271, 113.9289365)
3 2496 22.3141092 113.9293734 22.3152271 113.9289365 (22.3141092, 113.9293734) 9.065208216443396e-05
(2495, 2207, 2208, 128.82027637960374, 'gate', 22.3139258, 113.9287001, 22.3150209, 113.9282889)
4 2495 22.3139258 113.9287001 22.3150209 113.9282889 (22.3139258, 113.9287001) 0.0006138664757071951


In [14]:
StartEdge

(2496,
 2209,
 2210,
 132.090974445492,
 'gate',
 22.3141092,
 113.9293734,
 22.3152271,
 113.9289365,
 (22.3141092, 113.9293734),
 9.065208216443396e-05)

In [15]:
#The Algorithm
#Filter Nodes outside of the airport...

#Which way should we iterate through points? Must start at a gate and end at a gate or runway... Check if gate is at start or end

#Once first edge is found... Follow along a path with PrevMatchedEdge as a variable, and is current edge a junction?

#If Current Edge is a junction, initiate second Fuzzy rules
FinalNode = []
EdgePath = [StartEdge]            
for counter, item in enumerate(NewNode):
    print('\n\n')
    print((counter, item,'NEW NODE'))
    
    #At the start or end, a Gate must be present
    if counter == 0:
        continue
    else:
        PrevNode = NewNode[counter -1]
        NextNode = NewNode[counter +1]
        Next2Node = NewNode[counter +2]
    PrevX = PrevNode[0]
    PrevY = PrevNode[1]
    PointX = item[0]
    PointY = item[1]
    NextX = NextNode[0]
    NextY = NextNode[1]
    Next2X = Next2Node[0]
    Next2Y = Next2Node[1]
    
    #Following along a link - SMP1: Does the current Node link with the previous edge?
    EdgeID = StartEdge[0]
    EdgeStart = StartEdge[1]
    EdgeEnd = StartEdge[2]
    EdgeLength = StartEdge[3]
    EdgeType = StartEdge[4]
        
    StartX = StartEdge[5]
    StartY = StartEdge[6]
    EndX = StartEdge[7]
    EndY = StartEdge[8]
    print(PrevX, PrevY, StartX, StartY, EndX, EndY)
    (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
    (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
    (PrevStartX, PrevStartY, PrevEndX, PrevEndY, Direction) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
    if Direction == True:
        PrevEdgeStart = EdgeStart
        PrevEdgeEnd = EdgeEnd
    else:
        PrevEdgeStart = EdgeEnd
        PrevEdgeEnd = EdgeStart
    #Input Variables:
    Alpha = GetAlpha(PrevX, PrevY, PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    Beta = GetBeta(PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    DeltaDistance = GetDistanceChange(PrevX, PrevY, PointX, PointY, PrevStartX, PrevStartY, PrevEndX, PrevEndY)
    HeadingIncrement = GetHeadingIncrement(PrevX, PrevY, PointX, PointY, NextX, NextY)
    Z = SMP1FuzzyLogic(HeadingIncrement, HeadingIncrement, DeltaDistance, Alpha, Beta)
    print('ALPHA', Alpha,'BETA', Beta,'D', DeltaDistance,'Theta', HeadingIncrement, 'Z', Z)
    if Z > 30:
        FinalNode.append((IntX, IntY))
        if EdgePath[-1][0] == StartEdge[0]:
            print('PREVIOUS EDGE CHOSEN', StartEdge)
        else:
            EdgePath.append(StartEdge)
        continue
    #First Fuzzy Logic to check if current node matches with
    
    #If SMP-1 states it has passed previous Edge...
    #Search for 8 closest EdgeNodes to Node and highlight the Edges connected to them
    tree = spatial.KDTree(Nodes)
    HighlightedNodeIDs = list(tree.query(item, 15)[1])
    HighlightedNodes = []
    for n in HighlightedNodeIDs:
        HighlightedNodes.append(Nodes[n])  
    
    HighlightEdge = []
    for m in EdgesAndNodes:
        for i in HighlightedNodes:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEdge.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEdge.append(m)
        else:
            pass
    HighlightedEdges = list(set(HighlightEdge))

    
    #Iterate through the Edges for each Trajectory Node
    EdgesSuccess = []
    for index, edge in enumerate(HighlightedEdges):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(PointX, PointY, StartX, StartY, EndX, EndY, 0.0015) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (PrevIntX, PrevIntY) = NearestPointOnLine(PrevX, PrevY, StartX, StartY, EndX, EndY)
            (IntX, IntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            HaversineDistance = Haversine(PointX, PointY, (IntX, IntY)[0], (IntX, IntY)[1])
            
            #Check if Edge is being traversed in reverse or not
            (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(PrevX, PrevY, PointX, PointY, StartX, StartY, EndX, EndY)
            if Direction == True:
                NewEdgeStart = EdgeStart
                NewEdgeEnd = EdgeEnd
            else:
                NewEdgeStart = EdgeEnd
                NewEdgeEnd = EdgeStart
            
            print('NEW EDGE', edge)
            print('HAVERSINE', HaversineDistance*1000) 
            Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
            if counter == 0:
                pass
            
            
            #Distance Error
            NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
            print(NodePath)
            DistanceError = GetDistanceError(NodePath, NewEdgeEnd, PrevX, PrevY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, PointX, PointY, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)
            print(DistanceError, 'Distance Error!', NewEdgeEnd, 'NewEdgeEnd')
            
            #Connectivity
            Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
            print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
            #SMP2 Fuzzy Logic - Deciding the junction
            SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
            EdgesSuccess.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
            print(EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2)
            
        else:
            print('EDGE DISCARDED')
            pass
    #GET CONSEQUENT NODE HIGHLIGHTED EDGES
    HighlightedNodeIDs2 = list(tree.query(NextNode, 15)[1])
    HighlightedNodes2 = []
    for n in HighlightedNodeIDs2:
        HighlightedNodes2.append(Nodes[n])  
    
    HighlightEdge2 = []
    for m in EdgesAndNodes:
        for i in HighlightedNodes2:
            if m[5] == i[0] and m[6] == i[1]:
                HighlightEdge2.append(m)
            elif m[7] == i[0] and m[8] == i[1]:
                HighlightEdge2.append(m)
        else:
            pass
    HighlightedEdges2 = list(set(HighlightEdge2))

    #CHECK THE CONSEQUENT NODE for the same Junction
    print('CHECKING THE NEXT NODE', '\n')
    EdgesSuccess2 = []
    for index, edge in enumerate(HighlightedEdges2):
        EdgeID = edge[0]
        EdgeStart = edge[1]
        EdgeEnd = edge[2]
        EdgeLength = edge[3]
        EdgeType = edge[4]
        
        StartX = edge[5]
        StartY = edge[6]
        EndX = edge[7]
        EndY = edge[8]
        #If Edge is within radius r0 of Node, then return True
        if EdgeCircleSearch(NextX, NextY, StartX, StartY, EndX, EndY, 0.0015) == True:
        #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
            (PrevIntX, PrevIntY) = NearestPointOnLine(PointX, PointY, StartX, StartY, EndX, EndY)
            (IntX, IntY) = NearestPointOnLine(NextX, NextY, StartX, StartY, EndX, EndY)
            PDistance = GetPerpendicularDistance(NextX, NextY, (IntX, IntY)[0], (IntX, IntY)[1])
            HaversineDistance = Haversine(NextX, NextY, (IntX, IntY)[0], (IntX, IntY)[1])
            
            #Check if Edge is being traversed in reverse or not
            (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(PointX, PointY, NextX, NextY, StartX, StartY, EndX, EndY)
            if Direction == True:
                NewEdgeStart = EdgeStart
                NewEdgeEnd = EdgeEnd
            else:
                NewEdgeStart = EdgeEnd
                NewEdgeEnd = EdgeStart
            
            Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
            if counter == 0:
                pass
            
            #Distance Error
            NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
            DistanceError = GetDistanceError(NodePath, NewEdgeEnd, PointX, PointY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, NextX, NextY, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)
            
            #Connectivity
            Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
            print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
            #SMP2 Fuzzy Logic - Deciding the junction
            SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
            EdgesSuccess2.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
        else:
            print('EDGE DISCARDED')
            pass

    StartEdge1 = max(EdgesSuccess, key = lambda t: t[-1])
    StartEdge2 = max(EdgesSuccess2, key = lambda t: t[-1])
    
    if StartEdge1[0] == StartEdge2[0]:
        StartEdge = StartEdge1
    
    else:
        print('CHECKING THE NEXTNEXT NODE', '\n')
        #GET NEXT NEXT NODE HIGHLIGHTED EDGES
        HighlightedNodeIDs3 = list(tree.query(Next2Node, 15)[1])
        HighlightedNodes3 = []
        for n in HighlightedNodeIDs3:
            HighlightedNodes3.append(Nodes[n])  

        HighlightEdge3 = []
        for m in EdgesAndNodes:
            for i in HighlightedNodes3:
                if m[5] == i[0] and m[6] == i[1]:
                    HighlightEdge3.append(m)
                elif m[7] == i[0] and m[8] == i[1]:
                    HighlightEdge3.append(m)
            else:
                pass
        HighlightedEdges3 = list(set(HighlightEdge3))

            #Check the next next node
        EdgesSuccess3 = []
        for index, edge in enumerate(HighlightedEdges3):
            EdgeID = edge[0]
            EdgeStart = edge[1]
            EdgeEnd = edge[2]
            EdgeLength = edge[3]
            EdgeType = edge[4]

            StartX = edge[5]
            StartY = edge[6]
            EndX = edge[7]
            EndY = edge[8]
            #If Edge is within radius r0 of Node, then return True
            if EdgeCircleSearch(Next2X, Next2Y, StartX, StartY, EndX, EndY, 0.0015) == True:
            #Input Variables: Euclidian Distance: IntX and IntY would be the Node snapping in place
                (PrevIntX, PrevIntY) = NearestPointOnLine(NextX, NextY, StartX, StartY, EndX, EndY)
                (IntX, IntY) = NearestPointOnLine(Next2X, Next2Y, StartX, StartY, EndX, EndY)
                PDistance = GetPerpendicularDistance(Next2X, Next2Y, (IntX, IntY)[0], (IntX, IntY)[1])
                HaversineDistance = Haversine(Next2X, Next2Y, (IntX, IntY)[0], (IntX, IntY)[1])

                #Check if Edge is being traversed in reverse or not
                (NewStartX, NewStartY, NewEndX, NewEndY, Direction) = GetDirection2(NextX, NextY, Next2X, Next2Y, StartX, StartY, EndX, EndY)
                if Direction == True:
                    NewEdgeStart = EdgeStart
                    NewEdgeEnd = EdgeEnd
                else:
                    NewEdgeStart = EdgeEnd
                    NewEdgeEnd = EdgeStart

                Percentage = EdgeCovered(IntX, IntY, NewStartX, NewStartY, NewEndX, NewEndY)
                if counter == 0:
                    pass

                #Distance Error
                NodePath = nx.dijkstra_path(G, PrevEdgeEnd, edge[1])
                DistanceError = GetDistanceError(NodePath, NewEdgeEnd, NextX, NextY, PrevIntX, PrevIntY, PrevEndX, PrevEndY, Next2X, Next2Y, IntX, IntY, EdgesAndNodes, StartEdge, edge, FlightNodesList)

                #Connectivity
                Connectivity = GetConnectivity(PrevEdgeEnd, edge, AllNodeLinks)
                print(Connectivity, 'Connectivity', PrevEdgeEnd, 'PrevEdgeEnd')
                #SMP2 Fuzzy Logic - Deciding the junction
                SMP2 = SMP2FuzzyLogic(Connectivity, DistanceError, HaversineDistance*1000)
                EdgesSuccess3.append((EdgeID, EdgeStart, EdgeEnd, EdgeLength, EdgeType, StartX, StartY, EndX, EndY, (IntX,IntY), SMP2))
            else:
                print('EDGE DISCARDED')
                pass

        StartEdge3 = max(EdgesSuccess3, key = lambda t: t[-1])
            
        Path1 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge1[1])
        Path2 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge2[1])
        Path3 = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge3[1])
        if IsSublist(Path2, Path1) == True and IsSublist(Path3, Path2) == True:
            StartEdge = StartEdge1
        elif IsSublist(Path3, Path2) == True and IsSublist(Path2, Path1) == False:
            StartEdge = StartEdge2
        elif IsSublist(Path2, Path1) == True and IsSublist(Path3, Path2) == False:
            StartEdge = StartEdge2
        else:
            StartEdge = StartEdge3
    PathSegment = nx.dijkstra_path(G, PrevEdgeEnd, StartEdge[1])
    print(PathSegment)
    print(StartEdge, 'STARTEDGE')
    (NewIntX, NewIntY) = NearestPointOnLine(PointX, PointY, StartEdge[5], StartEdge[6], StartEdge[7], StartEdge[8])
    FinalNode.append((NewIntX, NewIntY))
    print(StartEdge, 'START EDGE', EdgePath[-1], 'LAST EDGE')
    if EdgePath[-1][0] == StartEdge[0]:
        print('PREVIOUS EDGE CHOSENNN')
    else:
        EdgePath.append(StartEdge)
        
    
    




(0, (22.314056, 113.9293), 'NEW NODE')



(1, (22.313953, 113.92933), 'NEW NODE')
22.314056 113.9293 22.3141092 113.9293734 22.3152271 113.9289365
ALPHA 5.107813821604594 BETA 143.1254499029909 D -0.00227030849783238 Theta 186.66692802828314 Z 69.0151394482441
PREVIOUS EDGE CHOSEN (2496, 2209, 2210, 132.090974445492, 'gate', 22.3141092, 113.9293734, 22.3152271, 113.9289365, (22.3141092, 113.9293734), 9.065208216443396e-05)



(2, (22.314095, 113.92927), 'NEW NODE')
22.313953 113.92933 22.3141092 113.9293734 22.3152271 113.9289365
ALPHA 1.559114206678533 BETA 4.9324578659800125 D 0.13042642717608444 Theta 180.2859047154648 Z 42.5
PREVIOUS EDGE CHOSEN (2496, 2209, 2210, 132.090974445492, 'gate', 22.3141092, 113.9293734, 22.3152271, 113.9289365, (22.3141092, 113.9293734), 9.065208216443396e-05)



(3, (22.314047, 113.92929), 'NEW NODE')
22.314095 113.92927 22.3141092 113.9293734 22.3152271 113.9289365
ALPHA 1.2732094912137484 BETA 105.36904400655752 D 0.005034404890452439 Theta 67.3801

3248.4099437430655 Distance Error! 2288 NewEdgeEnd
0 Connectivity 2209 PrevEdgeEnd
2541 2287 2288 137.67889012093454 gate 22.3130554 113.9289541 22.3119587 113.9295773 (22.3130554, 113.9289541) 5.333333333333333
NEW EDGE (1994, 374, 372, 19.338397843402362, 'taxiway', 22.3156208, 113.9292963, 22.3154518, 113.9292514)
HAVERSINE 156.35173707314
[2209, 2210, 1995, 372, 374]
341.3660347490939 Distance Error! 374 NewEdgeEnd
0 Connectivity 2209 PrevEdgeEnd
1994 374 372 19.338397843402362 taxiway 22.3156208 113.9292963 22.3154518 113.9292514 (22.3154518, 113.9292514) 5.333333333333333
NEW EDGE (974, 1997, 2208, 43.63864495444659, 'taxiway', 22.3148931, 113.9278875, 22.3150209, 113.9282889)
HAVERSINE 150.62181958262255
[2209, 2210, 2208, 1997]
395.50724254182035 Distance Error! 2208 NewEdgeEnd
0 Connectivity 2209 PrevEdgeEnd
974 1997 2208 43.63864495444659 taxiway 22.3148931 113.9278875 22.3150209 113.9282889 (22.3150209, 113.9282889) 5.333333333333333
NEW EDGE (2496, 2209, 2210, 132.090974445

2495 2207 2208 128.82027637960374 gate 22.3139258 113.9287001 22.3150209 113.9282889 (22.3139258, 113.9287001) 5.33334595028189
NEW EDGE (2543, 2291, 2292, 135.0328633602965, 'gate', 22.3134847, 113.9302818, 22.3123401, 113.930723)
HAVERSINE 115.06078069055538
[2209, 2210, 2208, 1997, 369, 2205, 371, 2203, 2353, 2201, 2198, 911, 2, 377, 2275, 936, 937, 2631, 376, 2273, 2270, 373, 890, 370, 835, 827, 804, 2623, 2622, 2621, 2620, 2619, 2618, 658, 634, 595, 570, 563, 1450, 1449, 1443, 2600, 2599, 2598, 2597, 605, 624, 2277, 649, 661, 2279, 2281, 2385, 2283, 2387, 2285, 755, 757, 759, 761, 2288, 2290, 775, 2292, 2291]
3359.7596139529564 Distance Error! 2291 NewEdgeEnd
0 Connectivity 2209 PrevEdgeEnd
2543 2291 2292 135.0328633602965 gate 22.3134847 113.9302818 22.3123401 113.930723 (22.3134847, 113.9302818) 5.333333333333333
NEW EDGE (978, 2212, 2214, 72.68722622805012, 'taxiway', 22.3154685, 113.9296821, 22.3156851, 113.9303493)
HAVERSINE 156.5479103819262
[2209, 2210, 1995, 2212]
366.1380

2495 2207 2208 128.82027637960374 gate 22.3139258 113.9287001 22.3150209 113.9282889 (22.3139258, 113.9287001) 5.333342960651744
NEW EDGE (2543, 2291, 2292, 135.0328633602965, 'gate', 22.3134847, 113.9302818, 22.3123401, 113.930723)
HAVERSINE 114.9714281870678
[2209, 2210, 2208, 1997, 369, 2205, 371, 2203, 2353, 2201, 2198, 911, 2, 377, 2275, 936, 937, 2631, 376, 2273, 2270, 373, 890, 370, 835, 827, 804, 2623, 2622, 2621, 2620, 2619, 2618, 658, 634, 595, 570, 563, 1450, 1449, 1443, 2600, 2599, 2598, 2597, 605, 624, 2277, 649, 661, 2279, 2281, 2385, 2283, 2387, 2285, 755, 757, 759, 761, 2288, 2290, 775, 2292, 2291]
3357.5938241525214 Distance Error! 2291 NewEdgeEnd
0 Connectivity 2209 PrevEdgeEnd
2543 2291 2292 135.0328633602965 gate 22.3134847 113.9302818 22.3123401 113.930723 (22.3134847, 113.9302818) 5.333333333333333
NEW EDGE (978, 2212, 2214, 72.68722622805012, 'taxiway', 22.3154685, 113.9296821, 22.3156851, 113.9303493)
HAVERSINE 155.89986799080052
[2209, 2210, 1995, 2212]
363.972

NEW EDGE (975, 2208, 2210, 70.40481673450411, 'taxiway', 22.3150209, 113.9282889, 22.3152271, 113.9289365)
HAVERSINE 133.93361450918553
[2210, 2208]
0.7576139434957798 Distance Error! 2210 NewEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
975 2208 2210 70.40481673450411 taxiway 22.3150209 113.9282889 22.3152271 113.9289365 (22.3152271, 113.9289365) 23.333177894062164
CHECKING THE NEXT NODE 

0 Connectivity 2210 PrevEdgeEnd
EDGE DISCARDED
EDGE DISCARDED
0 Connectivity 2210 PrevEdgeEnd
(22.3152271, 113.9289365) NodeLonLat
1 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
This is the previous edge yo
1 Connectivity 2210 PrevEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
0 Connectivity 2210 PrevEdgeEnd
1 Connectivity 2210 PrevEdgeEnd
[221

...

ALPHA 70.452312702978 BETA 36.63945874576061 D 0.025444373109285785 Theta 29.746892659492346 Z 23.33336431693041
NEW EDGE (1268, 1245, 1247, 27.276950249213268, 'taxiway', 22.2965162, 113.8987762, 22.2965757, 113.8990336)
HAVERSINE 30.170117294344777
[1242, 1247, 1245]
66.02105970797142 Distance Error! 1247 NewEdgeEnd
1 Connectivity 1242 PrevEdgeEnd
1268 1245 1247 27.276950249213268 taxiway 22.2965162 113.8987762 22.2965757 113.8990336 (22.2965162, 113.8987762) 28.266900614121138
EDGE DISCARDED
NEW EDGE (1184, 1947, 22, 67.57356387217351, 'taxiway', 22.297562, 113.8976031, 22.2969909, 113.8978289)
HAVERSINE 104.9542640397359
[1242, 1247, 1245, 22, 1947]
326.2708204256169 Distance Error! 22 NewEdgeEnd
0 Connectivity 1242 PrevEdgeEnd
1184 1947 22 67.57356387217351 taxiway 22.297562 113.8976031 22.2969909 113.8978289 (22.2969909, 113.8978289) 5.333333333333333
EDGE DISCARDED
NEW EDGE (9176, 1242, 10, 119.78806524, 'taxiway', 22.2964615, 113.898793, 22.295528, 113.8983679)
HAVERSINE 16.575

1 Connectivity 1242 PrevEdgeEnd
963 10 1952 87.4356510940375 taxiway 22.295528 113.8983679 22.2962729 113.8980939 (22.29620019930453, 113.8981206418319) 20.333333419776654
EDGE DISCARDED
NEW EDGE (9110, 1951, 1950, 139.585117175, 'taxiway', 22.2952903, 113.8984765, 22.2949395, 113.8996718)
HAVERSINE 137.7063003664608
[1242, 10, 1951]
272.07569216394427 Distance Error! 1950 NewEdgeEnd
0 Connectivity 1242 PrevEdgeEnd
9110 1951 1950 139.585117175 taxiway 22.2952903 113.8984765 22.2949395 113.8996718 (22.295279193216363, 113.89851434475052) 5.333333333333333
EDGE DISCARDED
NEW EDGE (846, 390, 389, 9.86388098468597, 'taxiway', 22.2979426, 113.8987035, 22.2978547, 113.8987169)
HAVERSINE 154.32464474642632
[1242, 1247, 1245, 22, 1947, 39, 41, 44, 389, 390]
546.882948727451 Distance Error! 390 NewEdgeEnd
0 Connectivity 1242 PrevEdgeEnd
846 390 389 9.86388098468597 taxiway 22.2979426 113.8987035 22.2978547 113.8987169 (22.2978547, 113.8987169) 5.333333333333333
NEW EDGE (9139, 1245, 22, 118.467

9076 380 387 83.2520861185 taxiway 22.2967416 113.8994247 22.2972696 113.898926 (22.2967416, 113.8994247) 4.752110473374493
NEW EDGE (706, 380, 381, 16.91159579354533, 'taxiway', 22.2967416, 113.8994247, 22.2967545, 113.8995886)
HAVERSINE 18.820432131762292
[1247, 1245, 22, 1947, 39, 41, 44, 389, 388, 387, 380]
601.9365849655844 Distance Error! 381 NewEdgeEnd
0 Connectivity 1247 PrevEdgeEnd
706 380 381 16.91159579354533 taxiway 22.2967416 113.8994247 22.2967545 113.8995886 (22.2967545, 113.8995886) 19.744562827621532
EDGE DISCARDED
NEW EDGE (698, 389, 388, 34.967002460391576, 'taxiway', 22.2978547, 113.8987169, 22.297556, 113.8988239)
HAVERSINE 130.40797992286969
[1247, 1245, 22, 1947, 39, 41, 44, 389]
468.21266882814774 Distance Error! 388 NewEdgeEnd
0 Connectivity 1247 PrevEdgeEnd
698 389 388 34.967002460391576 taxiway 22.2978547 113.8987169 22.297556 113.8988239 (22.297556, 113.8988239) 5.333333333333333
NEW EDGE (1573, 1242, 1247, 27.80166975421572, 'taxiway', 22.2964615, 113.89879

0 Connectivity 91 PrevEdgeEnd
EDGE DISCARDED
0 Connectivity 91 PrevEdgeEnd
EDGE DISCARDED
0 Connectivity 91 PrevEdgeEnd
CHECKING THE NEXTNEXT NODE 

0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
EDGE DISCARDED
EDGE DISCARDED
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
EDGE DISCARDED
EDGE DISCARDED
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
0 Connectivity 91 PrevEdgeEnd
EDGE DISCARDED
False
False
False
[91, 86, 77, 65, 61, 58, 50, 1836, 1813, 1821, 1831, 2518, 2517, 2516, 2515, 1662, 1663, 2514, 1959, 1574, 1577, 1580, 1593, 2513, 2512, 2511, 2007, 1607, 2510, 2509, 2508, 2507, 1202, 2506, 2505, 120, 2009, 1632, 1633, 1634, 136, 139, 141, 143, 14

ValueError: max() arg is an empty sequence

In [16]:
EdgePath

[(2496,
  2209,
  2210,
  132.090974445492,
  'gate',
  22.3141092,
  113.9293734,
  22.3152271,
  113.9289365,
  (22.3141092, 113.9293734),
  9.065208216443396e-05),
 (976,
  2210,
  1995,
  9.64146637951612,
  'taxiway',
  22.3152271,
  113.9289365,
  22.3152553,
  113.9290252,
  (22.315252711099937, 113.92901705689945),
  70.16406128851756),
 (974,
  1997,
  2208,
  43.63864495444659,
  'taxiway',
  22.3148931,
  113.9278875,
  22.3150209,
  113.9282889,
  (22.314951657045828, 113.92807141860872),
  49.83388940420932),
 (970,
  2203,
  371,
  67.69080836056689,
  'taxiway',
  22.3152993,
  113.9268148,
  22.3150318,
  113.9274064,
  (22.315285081577326, 113.92684624530415),
  35.18941270488256),
 (966,
  911,
  2198,
  31.60311868805605,
  'taxiway',
  22.3161241,
  113.9249911,
  22.3159992,
  113.9252673,
  (22.31601539614189, 113.92523148435235),
  35.225729962891265),
 (9081,
  936,
  2275,
  55.7934924543,
  'taxiway',
  22.3165426,
  113.923993,
  22.3163568,
  113.9244932,
  

In [17]:
FinalNode

[(22.3141092, 113.9293734),
 (22.31413193796317, 113.92936451350201),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.3141092, 113.9293734),
 (22.314642646468574, 113.92916491734314),
 (22.314858016246525, 113.92908074607918),
 (22.315249531372064, 113.92900705541497),
 (22.3152553, 113.9290252),
 (22.315252711099937, 113.92901705689945),
 (22.3150209, 113.9282889),
 (22.3150209, 113.9282889),
 (22.314951657045828, 113.92807141860872),
 (22.3150318, 113.9274064),
 (22.315127351139267, 113.927195080172),
 (22.315285081577326, 113.92684624530415),
 (22.3159992, 113.9252673),
 (22.3159992, 113.9252673),
 (22.31601539614189, 113.92523148435235),
 (22.3163568, 113.9244932),
 